<a href="https://colab.research.google.com/github/vineet96/vertex-ai-class/blob/main/object_detection_online_and_batch_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage     \
                         tensorflow -q


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)



In [ ]:
PROJECT_ID = "vertexai-dev-387420"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import google.cloud.aiplatform as aiplatform
aiplatform.init(project=PROJECT_ID)

In [ ]:
dataset = aiplatform.ImageDataset('4970933850617151488')

In [ ]:
#IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"



#DISPLAY_NAME = "salads_unique"
#dataset = aiplatform.ImageDataset.create(
 #   display_name=DISPLAY_NAME,
 #   gcs_source=[IMPORT_FILE],
#    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
#)

#print(dataset.resource_name)

In [ ]:
# dag = aiplatform.AutoMLImageTrainingJob(
#     display_name="saladprediction",
#     prediction_type="object_detection",
#     multi_label=False,
#     model_type="CLOUD",
#     base_model=None,
# )

# print(dag)

In [ ]:
# model = dag.run(
#     dataset=dataset,
#     model_display_name="saladprediction",
#     training_fraction_split=0.8,
#     validation_fraction_split=0.1,
#     test_fraction_split=0.1,
#     budget_milli_node_hours=8000,
#     disable_early_stopping=False,
# )

In [ ]:
model = aiplatform.Model("projects/1052777806387/locations/us-central1/models/1847436820384579584")

In [ ]:
endpoint = aiplatform.Endpoint('527092680216281088')

**Online prediction**

In [ ]:
test_image1 = "gs://vertexai-dev-salad-object-detection/salad1.jpeg"

# New Section

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_image1, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

In [ ]:
BUCKET_URI = "gs://vertexai-dev-salad-object-detection"


In [ ]:
test_image2 = "gs://vertexai-dev-salad-object-detection/salad2.jpeg"

In [ ]:
import json

import tensorflow as tf
gcs_input_uri = BUCKET_URI + "/test.jsonl"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_image1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_image2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="saladbatchrequest",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)


In [ ]:
batch_predict_job.wait()


In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

endpoint.undeploy_all()

endpoint.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML trainig job
dag.delete()

# Delete the batch prediction job
batch_predict_job.delete()
